In [2]:
#Loading LLM Model
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from langchain_groq import ChatGroq

llamaChatModel = ChatGroq(model="llama-3.3-70b-versatile")

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [6]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", 
    google_api_key=api_key
)

E0000 00:00:1759560307.587777 10415718 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [16]:
loader = TextLoader("./data/state_of_the_union.txt")
loaded_document = loader.load()

In [17]:
loaded_document


[Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n')]

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

Created a chunk of size 164, which is longer than the specified 100
Created a chunk of size 169, which is longer than the specified 100
Created a chunk of size 122, which is longer than the specified 100


In [23]:
len(chunks_of_text)

8

In [32]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=api_key )



E0000 00:00:1759562490.800524 10415718 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [33]:
# # ✅ Safer batching (manual control)
# def create_faiss_index_safe(docs, batch_size=5, sleep_time=2):
#     texts = [doc.page_content for doc in docs]
#     metadatas = [doc.metadata for doc in docs]

#     vectors = []
#     for i in range(0, len(texts), batch_size):
#         batch_texts = texts[i:i+batch_size]
#         print(f"🔹 Embedding batch {i} → {i+len(batch_texts)}")

#         try:
#             batch_embeddings = embeddings.embed_documents(batch_texts)
#         except Exception as e:
#             print("⚠️ Error while embedding batch:", e)
#             break

#         for text, vector, meta in zip(batch_texts, batch_embeddings, metadatas[i:i+batch_size]):
#             vectors.append((Document(page_content=text, metadata=meta), vector))

#         time.sleep(sleep_time)  # avoid rate-limit

#     # Build FAISS index manually
#     texts_final = [doc for doc, _ in vectors]
#     vectors_final = [vec for _, vec in vectors]

#     faiss_index = FAISS.from_embeddings(texts_final, vectors_final, embeddings)
#     return faiss_index


# # ⚡ Use this instead of FAISS.from_documents
# vector_db = create_faiss_index_safe(chunks_of_text, batch_size=5, sleep_time=2)

# print("✅ FAISS index created safely with Gemini embeddings")

# # vector_db = FAISS.from_documents(chunks_of_text, embeddings)